# 라이브러리 

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

# 하이퍼파라미터 설정
NFOLDS = 3
SEED = 0

[변경 사항]    
원본 코드가 옛날 코드라 바뀐 부분이 있음    
`from sklearn.cross_validation import KFold`   
-> `from sklearn.model_selection import KFold`   
이렇게 바뀜

# 데이터 
캐글 내 Home Credit Default Risk 데이터로 각 파일이 의미하는 바는 다음과 같음.
- application : 대출 신청 시 작성한 내용.
- Previous application : 과거 대출 기록.
- bureau : 개인신용평가기관에 기록된 신청자의 과거 타금융기관과 신용거래 내역(국내의 NICE / KCB).
- application_train.csv : 학습 메인 테이블.
- application_test.csv : 테스트 메인 테이블.
- bureau.csv : 신용평가기관에서 제공한 신용도 정보.
- bureau_balance.csv : 이전 신용거래 월 잔액 정보.
- POS_CASH_balance.csv : 신용 거래 정보.
- credit_card_balance.csv : 신용카드 월 잔액 정보.
- previous_application.csv : 이전 가계신용대출 정보.
- installments_payments.csv : 대출 상환 내역 정보.
- HomeCredit_columns_description.csv : 파일의 열에 대한 정보.

In [26]:
data = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
prev = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')

In [28]:
# Train 데이터 확인
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Test 데이터 확인
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## 데이터 전처리
`pd.factorize()` -> 문자열 카테고리를 정수로 인코딩해주는 함수   

In [31]:
# 데이터에서 범주형(categorical) 변수만 뽑기
categorical_feats = [
    f for f in data.columns if data[f].dtype=='object'
]

# 범주형 변수를 숫자로 인코딩하기 = label encoding
for f_ in categorical_feats:
    # indexer 안에는 카테고리 -> 숫자 매핑 정보 있음
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

gc.enable() #메모리 절약

# 학습을 위한 타켓값 만들기
y_train = data['TARGET']
del data['TARGET']


## previous_application 인코딩도 마찬가지의 방법으로 
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]

for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

## previous_application은 인코딩만 하는게 아니라 이걸 통해 평균 요약 테이블 만듦
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

근데 보면 원작성자가 
```
###################################
# PLEASE DON'T DO THIS AT HOME LOL
# Averaging factorized categorical features defeats my own reasoning
###################################
```
이렇게 경고를 적은걸 볼 수 있는데 이는 우리가 인코딩한 범주 간의 관계는 숫자 간의 관계로 해석될 수 없음에도 mean을 하면서 이런 오류가 생길 수 있기 때문임. => 원래는 이렇게 categorical을 숫자로 바꿔서 평균내면 안 됌.    
즉, 대충 "초보자면 따라하지마라" 이런 의미.   
그래서 GPT(진심 만능임)한테 그럼에도 왜 쓰냐고 물어보니까
- Home Credit 대회는 데이터가 매우 크고 복잡함 
- Tree 모델(Boosting, RF, ET)이 숫자를 선형적 관계로 보지 않음
- 캐글용 트릭
    -  높은 차원의 one-hot을 만들면 느림
    -  target encoding은 leakage 위험
    -  factorize+aggregation은 빠르고 간단하면서도 효과 있음            

이런 이유로 쓸 수도 있다고 한다

# 모델 